In [26]:
from data import data2

host = '192.168.70.1'
database = 'binance_1'
db = data2.data(host, database, True)
crypto_id = 1
start_time = '2024-01-01T00:00:00.000Z'
end_time = '2024-10-01T00:00:00.000Z'

df_prices = db.query_to_df(f"SELECT id, time_open, open, high, low, close, volume, trades, buy_volume, buy_quote_volume FROM price WHERE crypto_id = {crypto_id} AND time_open >= '{start_time}' AND time_open < '{end_time}' ORDER BY time_open")
display(df_prices.head())


c:\code\github\mikekoder\cryptotrader\predict\data\data2.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, cnxn)


,id,time_open,open,high,low,close,volume,trades,buy_volume,buy_quote_volume
0,135951,2024-01-01 00:00:00,42283.58,42554.57,42261.02,42475.23,1271.68108,47134,682.57581,2.895742e+07
1,1747917,2024-01-01 01:00:00,42475.23,42775.00,42431.65,42613.56,1196.37856,50396,712.32227,3.035565e+07
2,135952,2024-01-01 02:00:00,42613.57,42638.41,42500.00,42581.10,685.21980,29863,288.98864,1.230102e+07
3,135957,2024-01-01 03:00:00,42581.09,42586.64,42230.08,42330.49,794.80391,38620,356.37209,1.511300e+07
4,135958,2024-01-01 04:00:00,42330.50,42399.99,42209.46,42399.99,715.41760,36038,371.12012,1.570362e+07


In [27]:

df_features = db.query_to_df(f"SELECT * FROM price_feature WHERE price_id IN (SELECT id FROM price WHERE crypto_id = {crypto_id} AND time_open >= '{start_time}' AND time_open < '{end_time}') ORDER BY price_id")
display(df_features.head())


c:\code\github\mikekoder\cryptotrader\predict\data\data2.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, cnxn)


,id,price_id,feature_id,value
0,18046466,6703,666,NaN
1,18046465,6703,665,45277.623333
2,18046464,6703,664,44879.106667
3,18046463,6703,663,44640.553333
4,18046462,6703,662,44242.036667


In [28]:

df_features = df_features.pivot(index='price_id', columns='feature_id', values='value')
display(df_features.head())

feature_id,553,554,555,556,557,558,559,560,561,562,...,751,752,753,754,755,756,757,758,759,760
price_id,,,,,,,,,,,,,,,,,,,,,
6703,-98.9269,12.2740,35.3312,-110.5807,0.0,-65.1947,38.2842,0.3764,-333.3604,-10505.7691,...,452.90,0.592158,44090.295,0.017704,0.024404,NaN,1275.003024,0.576198,0.576198,1275.003024
6709,-185.7894,11.5166,35.8991,-225.8382,0.0,-82.9954,41.1817,-0.5660,-299.4295,-3720.7126,...,360.82,0.612673,44026.340,0.020635,0.025262,NaN,-1647.747090,-0.962197,-0.962197,-1647.747090
6710,15.0974,20.7563,31.5073,71.8117,0.0,52.1140,29.3847,0.9803,57.2123,527.3594,...,190.23,0.506771,43028.990,0.017244,0.020037,NaN,-230.974666,-0.303580,-0.303580,-230.974666
7762,-133.8507,22.9955,11.6656,-97.2999,0.0,48.5868,28.9419,-0.2931,62.2573,2991.6075,...,261.04,0.478123,43028.990,0.017244,0.018461,NaN,-1069.474800,-0.908060,-0.908060,-1069.474800
7838,-8.8676,20.9583,26.5948,44.3305,0.0,65.1763,29.3751,0.9606,-18.8774,-2742.4931,...,142.77,0.494263,43028.990,0.017244,0.019358,NaN,-247.791650,-0.338236,-0.338236,-247.791650


In [1]:
# merge prices and features
df = df_prices.merge(df_features, left_on='id', right_on='price_id', right_index=True)

display(df.head())

NameError: name 'df_prices' is not defined

In [30]:
#display(df.columns)

index = 2
for col in df.columns:
    if col != 'id' and col != 'time_open':
        df.insert(index, str(col) + '_prev', df[col].shift(1))
        index += 1

display(df.head()) 

: 

In [ ]:
# print all columns
for col in df.columns:
    print(col)

In [25]:
# shift output by 1
df['buy_price'] = df['low'].shift(-1)
df['sell_price'] = df['high'].shift(-1)
display(df.head())

C:\Users\mkode\AppData\Local\Temp\ipykernel_55124\2746630135.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['buy_price'] = df['low'].shift(-1)
C:\Users\mkode\AppData\Local\Temp\ipykernel_55124\2746630135.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['sell_price'] = df['high'].shift(-1)


,id,time_open,open_prev,high_prev,low_prev,close_prev,volume_prev,trades_prev,buy_volume_prev,buy_quote_volume_prev,...,753,754,755,756,757,758,759,760,buy_price,sell_price
0,135951,2024-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42477.5,0.019846,0.022823,NaN,584.267168,0.459445,0.459445,584.267168,42431.65,42775.00
1,1747917,2024-01-01 01:00:00,42283.58,42554.57,42261.02,42475.23,1271.68108,47134.0,682.57581,2.895742e+07,...,42477.5,0.019846,0.023767,NaN,71.326253,0.059618,0.059618,71.326253,42500.00,42638.41
2,135952,2024-01-01 02:00:00,42475.23,42775.00,42431.65,42613.56,1196.37856,50396.0,712.32227,3.035565e+07,...,42477.5,0.019846,0.022688,NaN,117.776021,0.171881,0.171881,117.776021,42230.08,42586.64
3,135957,2024-01-01 03:00:00,42613.57,42638.41,42500.00,42581.10,685.21980,29863.0,288.98864,1.230102e+07,...,42477.5,0.019846,0.023785,NaN,-347.158293,-0.436785,-0.436785,-347.158293,42209.46,42399.99
4,135958,2024-01-01 04:00:00,42581.09,42586.64,42230.08,42330.49,794.80391,38620.0,356.37209,1.511300e+07,...,42477.5,0.019846,0.023205,NaN,715.417600,1.000000,1.000000,715.417600,42180.77,42406.00
